Podan je slovar `postaje`, katerega ključi so imena postaj Biciklja, vrednosti pa slovarji z zemljepisnimi koordinatami postajališč, kapaciteto (število stojal) in trenutnim številom koles na postaji. (Glej slovar v testih.)

Podana je tudi funkcija `razdalje(postaja1, postaja2)`, ki prejme imeni postaj in vrne razdaljo med njima. 

Najprej, za vtis, rešitev celotnega izpita v obliki, kot bi si jo želel od študentov. Posamične naloge so razložene spodaj.

In [12]:
import json
from urllib.request import urlopen

import numpy as np

# 1

def prenesi():
    return {x["name"]:{
            "latitude": x["position"]["latitude"],
            "longitude": x["position"]["longitude"],
            "capacity": x["totalStands"]["capacity"],
            "bikes": x["totalStands"]["availabilities"]["mechanicalBikes"]}
           for x in json.loads(urlopen(URL).read())}

# 2

def najblizje(postaja):
    pari = []
    for ime in postaje:
        if ime != postaja:
            pari.append((razdalja(postaja, ime), ime))
    pari.sort()
    return pari[1:4]

# 3

def zapisi(ime_datoteke):
    f = open(ime_datoteke, "w", encoding="utf-8")
    for ime in sorted(postaje):
        f.write(f"{ime:40}{postaje[ime]['bikes']:2}/{postaje[ime]['capacity']:<2}\n")

# 4

def spremembe(prevozi):
    return np.sum(prevozi, axis=0) - np.sum(prevozi, axis=1)

def asimetrija(prevozi):
    r = np.abs(prevozi - prevozi.T)
    return (r == np.max(r)).nonzero()[0]

# 5

def spust_na_0(matrika, x, y):
    if matrika[y, x] == 0:
        return 1
    nicel = 0
    for nx, ny in ((x-1, y), (x+1, y), (x, y-1), (x, y+1)):
        if 0 <= nx < matrika.shape[1] and 0 <= ny < matrika.shape[0] and matrika[ny, nx] < matrika[y, x]:
            nicel += spust_na_0(matrika, nx, ny)
    return nicel

### 1. Branje podatkov [tema: json, slovarji]

Napišite funkcijo `prenesi()`, ki prebere podatke z naslova [https://ucilnica.fri.uni-lj.si/mod/resource/view.php?id=59478](https://ucilnica.fri.uni-lj.si/mod/resource/view.php?id=59478) (naslov je shranjen v spremenljivki URL v testih), jih predela v slovar v enaki obliki kot podani slovar postaje, in ga vrne. (Opomba: podatki so kopija resničnih, sproti osveževanih podatkov z naslova `https://api.jcdecaux.com/vls/v3/stations?apiKey=frifk0jbxfefqqniqez09tw4jvk37wyf823b5j1i&contract=ljubljana`, ki je med izpitom nedosegljiv.)

#### Rešitev

Potrebno je bilo pogledati, kaj je na tem naslovu in prepoznati obliko json. (Že za omembo jsona v rešitvi sem dal 5 točk. Večina študentov je žal nekaj brala in split-ala.) Rezultat je bil slovar, iz katerega je bilo potrebno le prestaviti podatke v nov, malo drugačen slovar.

In [1]:
from urllib.request import urlopen

def prenesi():
    return {x["name"]:{
            "latitude": x["position"]["latitude"],
            "longitude": x["position"]["longitude"],
            "capacity": x["totalStands"]["capacity"],
            "bikes": x["totalStands"]["availabilities"]["mechanicalBikes"]}
           for x in json.loads(urlopen(URL).read())}

Kdor na mara izpeljanih slovarjev in podobnih rokohitrstev, je šel lepo počasi.

In [2]:
def prenesi():
    t = urlopen(URL).read()
    postaje = {}
    for x in json.loads(t):
        postaje[x["name"]] = {
            "latitude": x["position"]["latitude"],
            "longitude": x["position"]["longitude"],
            "capacity": x["totalStands"]["capacity"],
            "bikes": x["totalStands"]["availabilities"]["mechanicalBikes"]}
    return postaje

### 2. Najbližje 3 [tema: seznami]

Avtor izpita ni uporabnik Biciklja (enkrat je bilo dovolj), ve pa, da na določeni postaji včasih ne moreš dobiti ali oddati kolesa, zato se je potrebno ozreti po bližnjih postajah.

Napišite funkcijo `najblizje3(postaja)`, ki za podano postajo poišče najbližje tri postaje in vrne seznam (treh) parov (razdalja, ime). Seznam naj bo urejen po razdaljah.

Klic `najblizje3("ŽIVALSKI VRT")` vrne `[(0.73687236710574, 'VIŠKO POLJE'), (0.9639016462683163, 'TEHNOLOŠKI PARK'), (1.0760703913614853, 'SOSESKA NOVO BRDO')]`.

#### Rešitev

Najpreprosteje je sestaviti seznam natančno takšnih parov, kot jih zahteva naloga (naloga je sestavljena, kakor je, prav zato, da vam da ta namig!), jih urediti in vrniti prve tri. (Tudi, da mora biti rezultat urejen, je nekaj, kar bi vas lahko napeljalo na pravo pot.)

In [3]:
def najblizje(postaja):
    pari = []
    for ime in postaje:
        if ime != postaja:
            pari.append((razdalja(postaja, ime), ime))
    pari.sort()
    return pari[1:4]

No, ne prvih treh: prvi element je vedno izhodiščna postaja in jo preskočimo, zato vzamemo `[1:4]`.

Še hitreje gre, če poznamo izpeljane sezname.

In [4]:
def najblizje3(postaja):
    return sorted((razdalja(postaja, name), name) for name in postaje)[1:4]

Če se ne domislimo urejanja, so stvari seveda bolj "zabavne". No, ni tako hudo: trikrat zapored poiščemo najbližjo postajo in jo dodamo v seznam.

In [5]:
def najblizje(postaja):
    moznosti = list(postaje)
    moznosti.remove(postaja)
    naj3 = []
    for i in range(3):
        naj = None
        for ime in moznosti:
            if naj is None or razdalja(postaja, ime) < razdalja(postaja, naj):
                naj = ime
        naj3.append((razdalja(postaja, naj), naj))
        moznosti.remove(naj)
    return naj3

Ni tako lepo, spet pa ni neka znanstvena fantastika.

### 3. Tabela postaj [tema: pisanje datotek in oblikovanje nizov]

Napišite funkcijo zapisi(ime_datoteke), ki v podano datoteko zapiše imena postaj (na 40 mest), ki ji sledi število razpoložljivih koles in število mest. Zapis mora biti urejen po abecedi imen postaj (šumniki pa bodo na koncu; uporabite običajno urejanje v Pythonu) in v spodnji obliki.

```
KONGRESNI TRG-ŠUBIČEVA ULICA             9/20
KOPALIŠČE ILIRIJA                        0/20
KOPALIŠČE KOLEZIJA                      19/20
KOPRSKA ULICA                            3/8 
KOSEŠKI BAJER                           14/20
```

#### Rešitev

Tole je bilo mišljeno kot podarjena naloga. Žal ste imeli nekaj nepredvidenih težav s poravnavanjem številk na desno in levo. Poglejte znakce za urejanje - `:2` in `:<2`.

In [6]:
def zapisi(ime_datoteke):
    f = open(ime_datoteke, "w", encoding="utf-8")
    for ime in sorted(postaje):
        f.write(f"{ime:40}{postaje[ime]['bikes']:2}/{postaje[ime]['capacity']:<2}\n")

### 4. Prevozi [tema: numpy]

Imamo matriko v numpyju, ki predstavlja podatke o prehodih koles med postajami v nekem časovnem obdobju: element z indeksoma `[i][j]` pove, koliko koles je bilo prevzetih na i-ti postaji in vrnjenih na j-to. Matrika na sliki nam pove, da so na postaji 1 prevzeli 3 kolesa, ki so jih vrnili na postajo 1; 7 koles, ki so jih vrnili na postajo 2; in 5 koles, ki so jih vrnili na 4.

- Skupno so s postaje 1 odpeljali 15 koles, pripeljali pa so jih (kje to vidimo?) 10. Na postaji 1 je pet koles manj, kot jih je bilo prej. Napišite funkcijo `spremembe(prevozi)`, ki prejme takšno matriko in vrne matriko s toliko elementi, kolikor je postaj. Vsak element pove, koliko koles več (pozitivna števila) oz. manj (negativna) kot prej je na postaji. Za matriko s slike vrne `[-6, -5, 12, -6, 9, -4]`.

- S postaje 1 na postajo 4 so pripeljali 5 koles, s postaje 4 na 1 pa samo 1 kolo. Razlika je torej 4. S postaje 4 na 5 so jih pripeljali 3, obratno prav tako 3. Razlika je 0. Največja razlika je med postajama 1 in 2: v eno smer so jih pripeljali 1, v drugo 7, razlika je 6. Napišite funkcijo `asimetrija(prevozi)`, ki vrne par postaj z največjo razliko. V gornjem primeru mora vrniti `[1, 2]` (ali `[2, 1]`). Če je parov z največjo razliko več, smete vrniti poljubnega.

Zaželeno je, da nalogo rešite s spretno uporabo numpyja – v vrstici ali dveh, brez kakšnih zank v Pythonu.

#### Rešitev

Kdor si je ogledal matriko, je videl, da i-ta vrstica pove, koliko koles je bilo prevzetih na i-ti postaji. Število prevzetih koles (po postajah) je torej `np.sum(prevozi, axis=1)`. Vsote stolpcev povedo, koliko koles je prišlo na posamično postajo. Rešitev je torej

In [7]:
def spremembe(prevozi):
    return np.sum(prevozi, axis=0) - np.sum(prevozi, axis=1)

Če zamešamo osi (kot sem jih prav med pisanjem tega besedila tudi sam), bodo imeli elementi rezultata ravno napačen predznak. Pač obrnemo.

Bistvo drugega dela je, da odštejemo elemente, ki so prezrcaljeni čez diagonalo, se pravi, recimo, `prevozi[2, 4]` od `prevozi[4, 2]`, kar storimo z `prevozi - prevozi.T`. Zanimajo nas le absolutne vrednosti, torej `np.abs(prevozi - prevozi.T)`. Že kdor je napisal to, je dobil veliko točk. Nekateri so nadaljevali z `argmax`, kar je načelno pravilna ideja, ki je prinesla nekaj točk, čeprav tule ne deluje. Tule je potrebno poiskati vse elemente, ki so enaki maksimum, nato `nonzero()`, da dobimo njihove indekse. Potem vzamemo prvega od njih.

In [8]:
def asimetrija(prevozi):
    r = np.abs(prevozi - prevozi.T)
    return (r == np.max(r)).nonzero()[0]

Ta naloga je bila zoprna zaradi `nonzero`; kot rečeno, sem nagrajeval tudi dobre ideje, ki v tem konkretnem primeru sicer niso delovale. Tudi rešitev v slogu

In [11]:
def asimetrija(prevozi):
    r = np.abs(prevozi - prevozi.T)
    m = np.max(r)
    for x in range(len(r)):
        for y in range(len(r)):
            if r[x][y] == m and x != y:
                return [x, y]  

nisem (preveč) zameril.

### 5. Spusti [tema: rekurzija]

Po nekem strašnem naključju predstavlja matrika na gornji sliki tudi nadmorske višine nekih točk na neki mreži. Kolesar vozi le navzdol: če se znajde na polju `[1][1]` (trojka) lahko gre gor (na 2), levo (na 0) ali dol (na 1), ne pa desno, ker je tam 7. Po diagonalah ne vozi.

Napišite funkcijo `spust_na_0(matrika, x, y)`, ki za podani x (stolpec) in y (vrstica) pove, koliko ničel je dosegljivih s tega polja. Če je možno do neke ničle priti na več načinov, jo štejte večkrat, ker je kolesar neumen in ne opazi, kadar pride do istega cilja samo po drugi poti.

Z omenjene trojke lahko pride na petih ničle: na eno neposredno, do dveh prek spodnje enke, če gre gor na dvojko, pa lahko gre od ondod levo ali desno na enki v prvi vrstici in potem na ničli zraven njiju. Ničlo v prvi vrstici štejemo dvakrat, ker jo dosežemo na dva načina.

Nasvet: ne vznemirjajte se zaradi "več načinov": naloga je takšna zato, da je lažja, ne težja. Če se boste "normalno" lotili naloge, boste večkratne poti že "ponesreči" šteli večkrat.

#### Rešitev

Tole je tako kot, recimo, velikost rodbine, le da so "potomci" sosednje celice z nižjo številko.

In [1]:
def spust_na_0(matrika, x, y):
    if matrika[y, x] == 0:
        return 1

    nicel = 0
    for nx, ny in ((x-1, y), (x+1, y), (x, y-1), (x, y+1)):
        if 0 <= nx < matrika.shape[1] and 0 <= ny < matrika.shape[0] and matrika[ny, nx] < matrika[y, x]:
            nicel += spust_na_0(matrika, nx, ny)
    return nicel